In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import importlib

import itertools as it
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.api as sm
import gc

from itertools import product
from sklearn.pipeline import make_pipeline

from criteriaetl.utils.expansion_func import (get_value_counts_with_expansion_factor,
    get_percentage_table_with_expansion_factor, get_weighted_average)
from criteriaetl.utils.display_func import cdisplay, percentage_count_plot
from criteriaetl.utils.common_func import (get_weighted_complete_randomization_series_on_subset, 
    proportional_cut, weighted_qcut, get_partition_bool_columns_dict)
from criteriaetl.transformers.columns_base import (NameTransformer, 
    ReplaceTransformer, SelectTransformer, AssignTransformer)
from criteriaetl.transformers.rows_base import AggregateTransformer
from criteriaetl.transformers.fusion_base import MergeTransformer

from projectetl.utils.dataload import (load_survey_data_do, save_survey_with_pickle,
                                       load_survey_from_pickle)
from projectetl.utils.config import (ENCFT_SURVEY_PATH, ENCFT_PREVIOUS_SURVEY_PATH,
                                     ENCFT_OBJECT_DIR, INFLATION_OBJECT_DIR, DATA_DIR,
                                     ENCFT_DIR)
from projectetl.utils.display import inspect_nulls
from projectetl.utils.save import protected_save
from projectetl.utils.prioritize import normalize_prioritization_variable

## Configuration variables

In [ ]:
# control flow
get_raw = 0 # if 1 will read the raw data, if 0 will load the data from a previously serialized pickle object
get_raw_incomes = 0
get_imputed = 1
get_icv = 1

# merge keys
key_member = ['trimestre', 'id_hogar', 'id_persona']
key_household = key_member[:-1]

# Load data

## Survey


In [ ]:
if get_raw:
    print('loading raw')
    survey_raw = load_survey_data_do(ENCFT_DIR / '2020 con pobreza calculada.csv', load_func=lambda path: pd.read_csv(str(path), low_memory=False))
    save_survey_with_pickle(survey_raw, ENCFT_OBJECT_DIR / 'encft2020.pkl')
else:
    print('loading pickle')
    survey_raw = load_survey_from_pickle(ENCFT_OBJECT_DIR / 'encft202001-202004.pkl')

In [ ]:
if get_icv:
    print('loading icv')
    survey_icv = load_survey_from_pickle(ENCFT_OBJECT_DIR / 'encft-2020-icv-household.pkl')

In [ ]:
if get_raw_incomes:
    print('loading csv')
    survey_incomes = load_survey_data_do(ENCFT_DIR / '2020 con pobreza calculada.csv', load_func=lambda path: pd.read_csv(str(path), low_memory=False))
    save_survey_with_pickle(survey_incomes, ENCFT_OBJECT_DIR / 'encft2020.pkl')
else:
    print('loading pickle')
    survey_incomes = load_survey_from_pickle(ENCFT_OBJECT_DIR / 'encft2020.pkl')
    
cols_from_incomes = key_member + ['ingpercapitadef', 'lpextrema', 'lpgeneral', 'quedateencasa', 
                                  'trsoc_def', 'trsoc', 'factor_expansion_anual']
survey_incomes_slim = survey_incomes[cols_from_incomes].copy()
del survey_incomes
gc.collect()

In [ ]:
if get_imputed:
    print('getting imputed')
    survey_member_raw = load_survey_from_pickle(ENCFT_OBJECT_DIR / 'encft-2020-use-time.pkl')
else:
    print('getting raw')
    survey_member_raw = survey_raw['Miembros']
    
survey_household_raw = survey_raw['Hogar']
survey_house_raw = survey_raw['Vivienda']

In [ ]:
cdisplay(survey_member_raw.head())

## Dictionary

In [ ]:
survey_dict = survey_raw['Diccionario']

In [ ]:
def consult_dict(field):
    return survey_dict[survey_dict['campo'] == field.upper()][['valor', 'descripcion', 'texto']] 

## Merge Transformer

In [ ]:
merge_survey_incomes_member_transformer = MergeTransformer(
    lambda cols=cols_from_incomes: survey_incomes_slim, merge_kwargs={'on': key_member,})
survey_member_merged = merge_survey_incomes_member_transformer.transform(
    survey_member_raw)

In [ ]:
survey_member_raw.shape, survey_member_merged.shape

## Replace Transformer

In [ ]:
replace_member_map = {
    'sexo': {
        1: 0,
        2: 1
    }
}

replace_member_transformer = ReplaceTransformer(replace_member_map)
survey_member_replaced = replace_member_transformer.transform(survey_member_merged)

## Assign Transformer

In [ ]:
assign_member_map = {
    # Segmentation
    'edad_menor_5': lambda df: (df['edad'] < 5).astype(int),
    'edad_mayor_60': lambda df: (df['edad'] > 60).astype(int),
    'edad_mayor_65': lambda df: (df['edad'] > 65).astype(int),
    'edad_5_a_10': lambda df: ((df['edad'] >= 5) & (df['edad'] < 10)).astype(int),
    'edad_10_a_15': lambda df: ((df['edad'] > 10) & (df['edad'] <= 15)).astype(int),
    # ---
    'es_jefe': lambda df: (df['parentesco'] == 1).astype(int),
    'es_conyugue': lambda df: (df['parentesco'] == 2).astype(int),
    'es_hijo': lambda df: (df['parentesco'] == 3).astype(int),
    'otro_pariente': lambda df: df['parentesco'].isin(range(4, 12)).astype(int),
    'no_pariente': lambda df: (df['parentesco'] == 12).astype(int),
    'sexo_jefe_familia': lambda df: df['sexo'] * df['es_jefe'],
    # ---
    'es_trabajador_domestico': lambda df: df['orden_sector'] == 3,
    'es_enfermo': lambda df: df['motivo_no_disponible'] == 4,
    'es_trabajador_infantil': lambda df: df['edad'].isin(range(0, 16)) & df['trabajo_semana_pasada'] == 1,
    # ---
    'es_informal': lambda df: df['orden_empleo'] == 2,
    'es_formal': lambda df: df['orden_empleo'] == 1, 
    'factor_anual': lambda df: df['factor_expansion'].div(4),
    
    # income cols
    'deflactor_trsoc': lambda df: df['trsoc_def'].div(df['trsoc']),
    'quedateencasa_def': lambda df: df['quedateencasa'].mul(df['deflactor_trsoc'])
}

time_use_assign_member_map = {
    'tiempo_no_remunerado': lambda df: df[[
        'tiempo_cuidados_horas_semanales', 
        'tiempo_trabajo_doméstico_no_remunerado_horas_semanales']].sum(1),
    'femenino_potencial_tnr': lambda df: df['sexo'].eq(1) & df['edad'].ge(10), 
    'tnr_femenino': lambda df:  df['tiempo_no_remunerado'].mul(df['femenino_potencial_tnr']), 
    'masculino_potencial_tnr': lambda df: df['sexo'].eq(0) & df['edad'].ge(10),
    'tnr_masculino': lambda df:  df['tiempo_no_remunerado'].mul(df['masculino_potencial_tnr']),
    
    
} if get_imputed else {}

assign_member_transformer = AssignTransformer({**assign_member_map, **time_use_assign_member_map})
survey_member_assigned = assign_member_transformer.transform(survey_member_replaced)

## Aggregate Transformer

In [ ]:
aggregate_member_map = {
    'sum': [
        'quedate_en_casa', #'ingreso_total', 'ingreso_subsidios',
        #'ingreso_monetario_exterior', 'ingreso_no_laboral_no_monetario',
        #'ingreso_no_laboral_monetario', 'ingreso_laboral_no_monetario',
        #'ingreso_laboral_monetario', 'ingreso_subsidios_covid',
        #'ingreso_sin_subsidios_covid', 'ingreso_sin_qec',
        'quedate_en_casa', 'comer_es_primero',
        'sexo_jefe_familia', 'edad_menor_5', 'edad_mayor_65',
        'es_jefe', 'es_conyugue', 'es_hijo', 'otro_pariente',
        'no_pariente', 'es_informal', 'es_formal', 
        'ingpercapitadef', 'quedateencasa_def', 
        
    ] + (['tnr_femenino', 'tnr_masculino', 'femenino_potencial_tnr', 
          'masculino_potencial_tnr'] if get_imputed else []),

    'any': [
        'edad_menor_5', 'edad_mayor_60', 'edad_mayor_65',
        'edad_5_a_10', 'edad_10_a_15', 'es_trabajador_domestico',
        'es_enfermo', 'es_informal', 'es_trabajador_infantil'
    ],
}
keep_first = ['factor_anual', 'miembro', 'cantidad_miembros_hogar', 'zona', 'grupo_region', #'trimestre'
              'lpgeneral', 'lpextrema'
             ]
aggregate_member_transformer = AggregateTransformer(aggregate_member_map, keep_first, key_household)
survey_household_member_aggregated = aggregate_member_transformer.transform(survey_member_assigned)

In [ ]:
survey_household_member_aggregated

# Processing Households

## Merge Transformer

In [ ]:

merge_survey_household_member_transformer = MergeTransformer(
    lambda : survey_household_raw, merge_kwargs={'on': key_household,})
merge_survey_household_icv_transformer = MergeTransformer(
    lambda : survey_icv, merge_kwargs={'on': key_household,})

survey_household_merged = make_pipeline( 
    merge_survey_household_member_transformer,
    merge_survey_household_icv_transformer,
).transform(survey_household_member_aggregated)

In [ ]:
survey_household_merged

## Replace Transformer

In [ ]:
household_replace_map = {
    'zona_first': {
        1: 0, # urbana
        2: 1 # rural
    }
}
household_replace_transformer = ReplaceTransformer(household_replace_map)
survey_household_replaced = household_replace_transformer.transform(survey_household_merged)

## Select Transformer

In [ ]:
estrato_region_map = {
    'DISTRITO NACIONAL': 'Región Ozama',
    'SANTO DOMINGO URBANO': 'Región Ozama',
    'SANTO DOMINGO RURAL': 'Región Ozama',
    'SANTIAGO URBANO': 'Región Cibao Norte',
    'SANTIAGO RURAL': 'Región Cibao Norte',
    'RESTO CIBAO NORTE URBANO': 'Región Cibao Norte',
    'RESTO CIBAO NORTE RURAL': 'Región Cibao Norte',
    'CIBAO SUR URBANO': 'Región Cibao Sur',
    'CIBAO SUR RURAL': 'Región Cibao Sur',
    ' CIBAO NORDESTE URBANO': 'Región Cibao Nordeste',
    ' CIBAO NORDESTE RURAL': 'Región Cibao Nordeste',
    ' CIBAO NOROESTE URBANO': 'Región Cibao Noroeste',
    ' CIBAO NOROESTE RURAL': 'Región Cibao Noroeste',
    ' VALDESIA URBANO': 'Región Valdesia',
    ' VALDESIA RURAL': 'Región Valdesia',
    ' EL VALLE URBANO': 'Región El Valle',
    ' EL VALLE RURAL': 'Región El Valle',
    ' ENRIQUILLO URBANO': 'Región Enriquillo',
    ' ENRIQUILLO RURAL': 'Región Enriquillo',
    ' HIGUAMO URBANO': 'Región Higuamo',
    ' HIGUAMO RURAL': 'Región Higuamo',
    ' YUMA URBANO': 'Región Yuma',
    ' YUMA RURAL': 'Región Yuma'
}

In [ ]:
select_household_map = {
    'region': {
        lambda df, estrato=estrato: df['des_estrato'] == estrato:
            lambda df, region=region: region
        for estrato, region in estrato_region_map.items()
    },

    'tipo_hogar': {
        lambda df: (df['es_jefe_sum'] == 1) & (
            df['es_conyugue_sum'] == 0) & (df['es_hijo_sum'] == 0) & (
            df['otro_pariente_sum'] == 0) & (df['no_pariente_sum'] == 0): "Unipersonal",
        lambda df: (df['es_jefe_sum'] == 1) & (
            df['es_conyugue_sum'] != 0) & (df['es_hijo_sum'] == 0) & (
            df['otro_pariente_sum'] == 0) & (df['no_pariente_sum'] == 0): "Nuclear sin hijos",
        lambda df: (df['es_jefe_sum'] == 1) & (
            df['es_conyugue_sum'] != 0) & (df['es_hijo_sum'] != 0) & (
            df['otro_pariente_sum'] == 0) & (df['no_pariente_sum'] == 0): "Nuclear con hijos",
        lambda df: (df['es_jefe_sum'] == 1) & (
            df['es_conyugue_sum'] == 0) & (df['es_hijo_sum'] != 0) & (
            df['otro_pariente_sum'] == 0) & (df['no_pariente_sum'] == 0): "Nuclear monoparental",
        lambda df: (df['es_jefe_sum'] == 1) & (df['es_conyugue_sum'] != 0)
            & (df['es_hijo_sum'] == 0) & (df['otro_pariente_sum'] != 0)
            & (df['no_pariente_sum'] == 0): "Extendido base nuclear sin hijos",
        lambda df: (df['es_jefe_sum'] == 1) & (df['es_conyugue_sum'] != 0) & (
            df['es_hijo_sum'] != 0) & (df['otro_pariente_sum'] != 0) & (
            df['no_pariente_sum'] == 0): "Extendido base nuclear con hijos",
        lambda df: (df['es_jefe_sum'] == 1) & (
            df['es_conyugue_sum'] == 0) & (df['es_hijo_sum'] != 0) & (
            df['otro_pariente_sum'] != 0) & (df['no_pariente_sum'] == 0): "Extendido base monoparental",
        lambda df: (df['es_jefe_sum'] == 1) & (
            df['es_conyugue_sum'] == 0) & (df['es_hijo_sum'] == 0) & (
            df['otro_pariente_sum'] != 0) & (df['no_pariente_sum'] == 0): "Extendido sin base nuclear",
        lambda df: (df['es_jefe_sum'] == 1) & (df['no_pariente_sum'] != 0): "Compuesto",
    },

    'informalidad_mayoria': {
        lambda df: df['es_informal_sum'] > df['es_formal_sum'] : True,
        'default': False
    }
}

select_household_transformer = SelectTransformer(select_household_map)
survey_household_selected = select_household_transformer.transform(survey_household_replaced)

In [ ]:
survey_household_selected

In [ ]:
inspect_nulls(survey_household_selected)

## Assign Transformer

In [ ]:
def generate_poverty(suffix, income_per_capita_col, lp='linea_pobreza', li='linea_pobreza_extrema'):
    return {
        # General Poverty
        f'pobreza_extrema{suffix}': lambda df, income_col=income_per_capita_col, li=li: (df[
            income_col] <= df[li]).astype(int),
        # Extreme Poverty
        f'pobreza_general{suffix}': lambda df, income_col=income_per_capita_col, lp=lp: (df[
            income_col] <= df[lp]).astype(int),
        f'pobreza{suffix}': lambda df: df[f'pobreza_extrema{suffix}'] + df[f'pobreza_general{suffix}']
    }


In [ ]:
n_buckets, n_buckets_total = 10, 65

assign_household_per_capita_map = {
    # Incomes corrected
    'ingreso_sin_qec_sum': lambda df: df['ingpercapitadef_sum'].subtract(df['quedateencasa_def_sum']),
    
    # Incomes per capita
    'ingreso_per_capita': lambda df: df['ingpercapitadef_sum'] / df['cantidad_miembros_hogar_first'],
    'ingreso_per_capita_sin_qec': lambda df: (df['ingreso_sin_qec_sum'] / df['cantidad_miembros_hogar_first']).fillna(0),
    'ingreso_per_capita_qec': lambda df: df['quedate_en_casa_sum'] / df['cantidad_miembros_hogar_first'], 
    
    # poverty lines
    'linea_pobreza': lambda df: df['lpgeneral_first'],
    'linea_pobreza_extrema': lambda df: df['lpextrema_first'],
    'linea_pobreza_norm': lambda df: df['linea_pobreza'] / df['linea_pobreza_extrema'],
    
    # Income Bins and normalize
    'ingreso_per_capita_bins': lambda df: proportional_cut(
        df, 'ingreso_per_capita', 'linea_pobreza', n_buckets, n_buckets_total, 
        include_lowest=True),
    'ingreso_sin_qec_sum_norm': lambda df: normalize_prioritization_variable(
        df, 'ingreso_per_capita', 'linea_pobreza', n_buckets, (1, n_buckets_total)),
    'ingreso_per_capita_sin_qec_bins': lambda df: proportional_cut(
        df, 'ingreso_per_capita_sin_qec', 'linea_pobreza', n_buckets, n_buckets_total,
        include_lowest=True, collapse_negative=True),
    'ingreso_per_capita_sin_qec_norm': lambda df: normalize_prioritization_variable(
        df, 'ingreso_per_capita_sin_qec', 'linea_pobreza', n_buckets, (1, n_buckets_total)),
    
    # Poverty
    **generate_poverty('', 'ingreso_per_capita'),
    #**generate_poverty('_sin_subsidio', 'ingreso_sin_subsidio_covid_per_capita'),
    **generate_poverty('_sin_qec', 'ingreso_per_capita_sin_qec'),
    
    # Cost Factors
    'factor_cost': lambda df: float(1.0),
    'factor_cost_percent': lambda df: (df['ingreso_per_capita_qec'] / 100.0).astype(float),
    
    # Impact Factors
    'factor_impact': lambda df, n_buckets=n_buckets: (n_buckets / df['linea_pobreza']).astype(float),
    'factor_impact_percent': lambda df: (df['factor_cost_percent'] * df['factor_impact']).astype(float),
    'factor_impact_dop': lambda df, n_buckets=n_buckets: (n_buckets / (df['linea_pobreza'] * df[
        'cantidad_miembros_hogar_first'])).astype(float),
    
    # Segmentation
    'recibe_qec': lambda df: df['quedateencasa_def_sum'] > 0.0,
    
    # Statistics
    'beneficio_promedio_qec': lambda df: 100.0 * df['ingreso_per_capita_qec'] / df['linea_pobreza'],
}

time_use_assign_household_map = {
    'tnr_femenino_promedio_hogar': lambda df: np.where(df[
        'femenino_potencial_tnr_sum'] > 0, df['tnr_femenino_sum'].div(df[
        'femenino_potencial_tnr_sum']), 0),
    'tnr_masculino_promedio_hogar': lambda df: np.where(df[
        'masculino_potencial_tnr_sum'] > 0, df['tnr_masculino_sum'].div(df[
        'masculino_potencial_tnr_sum']), 0),
} if get_imputed else {}
    
assign_household_transformer = AssignTransformer({**assign_household_per_capita_map, **time_use_assign_household_map})
survey_household_assigned_per_capita = assign_household_transformer.transform(survey_household_selected)

In [ ]:
inspect_nulls(survey_household_assigned_per_capita)

In [ ]:
sns.countplot(x='ingreso_per_capita_bins', data=survey_household_assigned_per_capita)

In [ ]:
sns.countplot(x='ingreso_per_capita_sin_qec_bins', data=survey_household_assigned_per_capita)

In [ ]:
"""
En Covid bajo la lupa, para junio 2020 se reportan los siguientes valores de pobreza extrema y pobreza general
gracias a los programas de mitigación:
* pobreza extrema - 5.1 % (6.39% corrected)
* pobreza general - 27.4 % (28.78% corrected)
"""
pov_table = get_percentage_table_with_expansion_factor(survey_household_assigned_per_capita, 'pobreza', 'factor_anual_first')
display(pov_table)
f"tasa de pobreza general: {pov_table.loc[[1, 2], 'percentage'].sum().round(1):.1f}"

In [ ]:
pov_table = get_percentage_table_with_expansion_factor(survey_household_assigned_per_capita, 'pobreza_sin_qec', 'factor_anual_first')
display(pov_table)
f"tasa de pobreza general: {pov_table.loc[[1, 2], 'percentage'].sum().round(1):.1f}"

In [ ]:
get_weighted_average(survey_household_assigned_per_capita, 'tnr_femenino_promedio_hogar', 'factor_anual_first')

In [ ]:
get_weighted_average(survey_household_assigned_per_capita, 'tnr_masculino_promedio_hogar', 'factor_anual_first')

# Pipeline

In [ ]:
survey_pipeline_processor = make_pipeline(
    merge_survey_incomes_member_transformer,
    replace_member_transformer,
    assign_member_transformer,
    aggregate_member_transformer,
    merge_survey_household_member_transformer,
    merge_survey_household_icv_transformer,
    # merge_survey_household_correction_transformer,
    household_replace_transformer,
    select_household_transformer,
    assign_household_transformer
)

survey_final_processed = survey_pipeline_processor.transform(survey_member_raw)

# Processing to Criteria

## Replace Transformer

In [ ]:
replace_final_map = {
    'sexo_jefe_familia_sum': {
        0: 'Hombre',
        1: 'Mujer',
    },
    'ps_comer_es_primero': {
        1: 'Beneficiario CEP',
        2: 'No Beneficiario CEP'
    },
    'recibe_qec': {
        True: 'Beneficiario QEC',
        False: 'No Beneficiario QEC'
    },
    'zona_first': {
        0: 'Urbana',
        1: 'Rural'
    },
    'edad_menor_5_any': {
        True: 'Con menores de 5',
        False: 'Sin menores de 5'
    },
    'edad_mayor_60_any': {
        True: 'Con mayores de 60',
        False: 'Sin mayores de 60'
    },
    'edad_mayor_65_any': {
        True: 'Con mayores de 65',
        False: 'Sin mayores de 65'
    },
    'edad_5_a_10_any': {
        True: 'Con miembros de 5 a 10',
        False: 'Sin miembros de 5 a 10'
    },
    'edad_10_a_15_any': {
        True: 'Con miembros de 10 a 15',
        False: 'Sin miembros de 10 a 15'
    },
    'es_trabajador_domestico_any': {
        True: 'Con trabajador doméstico',
        False: 'Sin trabajador doméstico'
    },
    'es_informal_any': {
        True: 'Con trabajador informal',
        False: 'Sin trabajador informal'
    },
    'tipo_hogar':{
        'Unipersonal': 'Unipers./Pareja sin hijos',
        'Nuclear sin hijos': 'Unipers./Pareja sin hijos',
        'Nuclear con hijos': 'Nuclear',
        'Nuclear monoparental': 'Monoparental no extendido',
        'Extendido base nuclear sin hijos': 'Extendido',
        'Extendido base nuclear con hijos': 'Extendido',
        'Extendido base monoparental': 'Extendido',
        'Extendido sin base nuclear': 'Extendido',
        'Compuesto': 'Compuesto',
    },
    'informalidad_mayoria': {
        True: 'Mayoría empleo informal',
        False: 'Minoría empleo informal'
    },
    'es_trabajador_infantil_any': {
        True: 'Con trabajo infantil',
        False: 'Sin trabajo infantil'
    },
    'es_enfermo_any': {
        True: 'Con inactivos por enfermedad',
        False: 'Sin inactivos por enfermedad'
    }
}

replace_final_transformer = ReplaceTransformer(replace_final_map)
survey_final_replaced = replace_final_transformer.transform(survey_final_processed)

## Name Transformer

In [ ]:
name_final_map = {
    'id_hogar': 'uid',
    'factor_anual_first': 'weight',
    # Segmentation
    'icv_basico_cat_first': 'Categorización Básica del SIUBEN II', 
    'zona_first': 'Zona Urbano-Rural',
    'ps_comer_es_primero': 'Beneficiario de Comer es Primero',
    'recibe_qec': 'Beneficiario de Quédate en Casa',
    'sexo_jefe_familia_sum': 'Sexo jefe de familia',
    'grupo_region': 'Macroregión geográfica',
    'region': 'Región geográfica',
    'tipo_hogar': 'Tipo familia',
    'edad_menor_5_any': 'Miembro menor de 5 años',
    'edad_mayor_60_any': 'Miembro mayor de 60 años',
    'edad_5_a_10_any': 'Miembro entre 5 y 10 años',
    'edad_10_a_15_any': 'Miembro entre 10 y 15 años',
    'edad_mayor_65_any': 'Miembro mayor de 65 años',
    'es_trabajador_domestico_any': 'Miembro trabajador doméstico',
    'es_informal_any': 'Miembro con empleo informal',
    'informalidad_mayoria': 'Mayoría de miembros con empleo informal',
    'es_enfermo_any': 'Miembro inactivo por enfermedad',
    # Statistics
    'edad_menor_5_sum': 'stat_edad_menor_5_count',
    'edad_mayor_65_sum': 'stat_edad_mayor_65_count',
    'cantidad_miembros_hogar_first': 'stat_miembros_hogar_count',
    'beneficio_promedio_qec': 'stat_beneficio_promedio_qec',
    'tnr_masculino_promedio_hogar': 'stat_tnr_promedio_masculino',
    'tnr_femenino_promedio_hogar': 'stat_tnr_promedio_femenino',
    
    # Factors
    'factor_cost': '(C) Beneficio en DOP',
    'factor_cost_percent': '(C) Beneficio en % del monto de QEC',
    'factor_impact': '(I) Beneficio en DOP',
    'factor_impact_dop': '(I) Beneficio en DOP (miembros)',
    'factor_impact_percent': '(I) Beneficio en % del monto de QEC',
    # Prioritization
    'ingreso_per_capita_sin_qec_norm': 'Ingreso del hogar antes de recibir QEC',
}

name_final_transformer = NameTransformer(name_final_map, keep_features=[])
survey_final_named = name_final_transformer.transform(survey_final_replaced)

In [ ]:
inspect_nulls(survey_final_named)

In [ ]:
cdisplay(survey_final_named.head())

# Export file

In [ ]:
protected_save(
    lambda df, path: df.to_csv(path, index=False), survey_final_named,
    f"{DATA_DIR}/exports_platform/QEC_escenario_s13.1.csv"
)